# Ev Fiyat Tahmini 

In [ ]:
#import some necessary librairies
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

color = sns.color_palette()
sns.set_style('darkgrid')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/..."
ROOT_DIR = "https://raw.githubusercontent.com/yavuzKomecoglu/yapay-zeka-egitimi-tubitak-bideb-2237a/main/"
DATASET_PATH = ROOT_DIR + "datasets/house_prices/"

In [ ]:
#Now let's import and put the train and test datasets in  pandas dataframe

train = pd.read_csv(DATASET_PATH+'train.csv')
test = pd.read_csv(DATASET_PATH+'test.csv')


In [ ]:
##display the first five rows of the train dataset.
train.head(5)

In [ ]:
##display the first five rows of the test dataset.
test.head(5)


In [ ]:
 #check the numbers of samples and features
print("The train data size before dropping Id feature is : {} ".format(train.shape))
print("The test data size before dropping Id feature is : {} ".format(test.shape))

#Save the 'Id' column
train_ID = train['Id']
test_ID = test['Id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

#check again the data size after dropping the 'Id' variable
print("\nThe train data size after dropping Id feature is : {} ".format(train.shape)) 
print("The test data size after dropping Id feature is : {} ".format(test.shape))

#Veri İşleme

##Aykırı Değerler

**GrLivArea**: Metrekare bazında yaşam alanı

In [ ]:
fig, ax = plt.subplots()

ax.scatter(x = train['GrLivArea'], y = train['SalePrice'])

plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)

plt.show()

Sağ altta son derece büyük `GrLivArea` ile düşük fiyatlı ikisini görebiliriz. Bu değerler gürültü olarak tanımlanır.

In [ ]:
#Deleting outliers
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

#Check the graphic again
fig, ax = plt.subplots()

ax.scatter(train['GrLivArea'], train['SalePrice'])

plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)

plt.show()

##Hedef Değişken

**SalePrice**, tahmin etmemiz gereken değişkendir. Öyleyse önce bu değişken üzerinde biraz analiz yapalım.

In [ ]:
##`scipy.stats` içerisindeki `norm` sınıfını kullanarak eğitim kümesindeki "SalePrice" dağılımının grafiğini çizdiriniz.


# normalize the exponential data with boxcox
normalized_data = stats.boxcox(train["SalePrice"])

# plot both together to compare
fig, ax=plt.subplots(1, 2, figsize=(15, 3))

sns.histplot(train["SalePrice"], ax=ax[0], kde=True, legend=False)
ax[0].set_title("Original Data")

sns.histplot(normalized_data[0], ax=ax[1], kde=True, legend=False)
ax[1].set_title("Normalized data")
plt.show()

In [ ]:
normalized_data

In [ ]:
sns.distplot(train['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)
plt.show()

In [ ]:
normalized_data

In [ ]:
len(normalized_data[0])

In [ ]:
train['SalePrice'] = normalized_data[0]
train

Hedef değişken sağa çarpık. 
(Doğrusal) modeller normal dağılımlı verileri sevdiğinden, bu değişkeni dönüştürmemiz ve daha normal dağılımlı hale getirmemiz gerekiyor.

**Hedef değişkenin günlük dönüşümü**

In [ ]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
train["SalePrice"] = np.log1p(train["SalePrice"])

#Check the new distribution 
sns.distplot(train['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Get also the QQ-plot
fig = plt.figure()
res = stats.probplot(train['SalePrice'], plot=plt)
plt.show()


Eğrilik şimdi düzeltilmiş görünüyor ve veriler daha normal dağılmış görünüyor.

##Öznitelik Mühendisliği

Önce train ve test dataframe'lerini birleştirerek tek bir dataframe haline getirelim

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]

y_train = train.SalePrice.values

all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)

print("all_data size is : {}".format(all_data.shape))


###Kayıp Veri (Missing Data)

In [ ]:
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100

all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]

missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

In [ ]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation=45, ha='right')

sns.barplot(x=all_data_na.index, y=all_data_na)

plt.xlabel('Features', fontsize=15)
plt.ylabel('Percent of missing values', fontsize=15)
plt.title('Percent missing data by feature', fontsize=15)

**Veri Korelasyonu**

In [ ]:
corrmat = train.corr()
corrmat

In [ ]:
#Correlation map to see how features are correlated with SalePrice
corrmat = train.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=0.9, square=True)

###Eksik Değerleri Tamamlama

- **PoolQC** : verisinin NA olması "Havuz Yok" anlamına geldiğini söylüyor. 
Çok büyük bir kayıp değer oranı (+%99) göz önüne alındığında bu mantıklıdır ve evlerin çoğunda genel olarak hiç Havuz yoktur.

In [ ]:
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")

- **MiscFeature** : verisinin NA olması "çeşitli özellik yok" anlamına geldiğini söylüyor.

In [ ]:
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")

- **Alley** : verisinin NA olması "ara sokak erişimi yok" anlamına geldiğini söylüyor.

In [ ]:
all_data["Alley"] = all_data["Alley"].fillna("None")

- **Fence(Çit)** : verisinin, NA olması "çit yok" anlamına geldiğini söylüyor.

In [ ]:
all_data["Fence"] = all_data["Fence"].fillna("None")

- **FireplaceQu** : verisinin NA olması "şömine yok" anlamına geldiğini söylüyor.

In [ ]:
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")

- **LotFrontage** : Evin sokakta kapladığı alan, eğer bu değer eksikse sokaktaki diğer evlerin kapladığı alanın ortanca (median) değeri ile doldurulabilir.

In [ ]:
all_data.groupby("Neighborhood")["LotFrontage"]

In [ ]:
#Mahalleye göre gruplandır ve eksik değeri tüm mahallenin ortanca LotFrontage'ına göre doldur
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

- **GarageType, GarageFinish, GarageQual ve GarageCond** : Eksik verileri Yok(None) ile değiştirme

In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')

- **GarageYrBlt, GarageArea ve GarageCars** : Eksik verilerin 0 ile değiştirilmesi (Garaj alanı yoksa garaj olmadığı için araç da yoktur.)


In [ ]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)

- **BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath ve BsmtHalfBath** : bodrum olmadığı için eksik değerler muhtemelen sıfırdır.

In [ ]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

- **BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1 ve BsmtFinType2** : Tüm bu kolonlar bodrumla ilgili özniteliklerin NaN değerinde olması bodrum olmadığı anlamına gelir.

In [ ]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

- **MasVnrArea ve MasVnrType** : NA büyük olasılıkla bu evler için duvar kaplaması olmadığı anlamına gelir. Alan için 0, tip için None doldurabiliriz.

In [ ]:
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

- **MSZoning (Genel imar sınıflandırması)** : 'RL' açık ara en yaygın değerdir. Böylece eksik değerleri 'RL' ile doldurabiliriz.

Not: Bir dizi değerin modu, en sık görünen değerdir. Birden çok değer olabilir.

In [ ]:
all_data['MSZoning'].value_counts()

In [ ]:
all_data['MSZoning'].mode()

In [ ]:
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

- **Utilities** : Bu kategorik özellik için bir "NoSeWa" ve 2 NA hariç tüm kayıtlar "AllPub" şeklindedir. 

'NoSewa' içeren ev eğitim setinde olduğundan, **bu özellik tahmine dayalı modellemeye yardımcı olmaz**. Daha sonra güvenle kaldırabiliriz.

In [ ]:
all_data = all_data.drop(['Utilities'], axis=1)

Functional: Ev işlevsellik derecesi
- **Functional** : Verisinin NA olması evin tipik bir ev olduğu anlamına geldiğini söylüyor.

In [ ]:
all_data["Functional"].value_counts()

In [ ]:
all_data["Functional"] = all_data["Functional"].fillna("Typ")

- **Electrical** : Bir adet NA değerine sahiptir. Bu öznitelik çoğunlukla 'SBrkr' olduğundan, bu değer ile değiştirebiliriz.

In [ ]:
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])

- **KitchenQual**: Yalnızca bir NA değeri ve Electrical ile aynı, KitchenQual'da eksik değer için 'TA' (en sık görülen) belirledik.

In [ ]:
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])

- **Exterior1st and Exterior2nd** : İki öznitelik içinde yalnızca bir eksik değer olduğundan en yaynın değeri (mode()) atayabiliriz.


In [ ]:
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])

- **SaleType** : En sık kullanılan "WD" ile doldurabiliriz.

In [ ]:
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])

- **MSSubClass** : Na büyük olasılıkla Bina sınıfı yok anlamına gelir. Eksik değerleri Yok(None) ile değiştirebiliriz

In [ ]:
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")

Kalan eksik değer var mı?

In [ ]:
#Check remaining missing values if any 
all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

Eksik değer kalmadı.

**Sahte kategorik özellikler elde etme**

In [ ]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

In [ ]:
all_data

Getting the new train and test sets. 

In [ ]:
train.shape

In [ ]:
train = all_data[:ntrain]
test = all_data[ntrain:]

#Modelleme

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

Sklearn'in **cross_val_score** fonksiyonunu kullanıyoruz. Ancak bu fonksiyonun karıştırma özelliği olmadığından öncesinde KFold ile karıştırıyoruz.

In [ ]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model, train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

**XGBoost** :

In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)



###Model Skoru

Çapraz doğrulama rmsle hatasını değerlendirerek modelin veriler üzerinde nasıl performans gösterdiğini görelim

In [ ]:
score = rmsle_cv(model_xgb)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score